In [1]:
import os
import json
import torch
torch.set_grad_enabled(False)
import numpy as np
from pprint import pprint
import sys
sys.path.append('../..')
from nebula.models.attention import TransformerEncoderModel
from nebula import JSONTokenizerBPE, PEDynamicFeatureExtractor
from nebula.misc import get_path, clear_cuda_cache, set_random_seed
from nebula.constants import *
from bertviz import model_view, head_view
from collections import Counter

SCRIPT_PATH = get_path(type="notebook")
ROOT = os.path.join(SCRIPT_PATH, "..", "..")

In [2]:
def get_attn(x, model):
    attentions = []
    x = model.pos_encoder(model.encoder(x))
    for layer in model.transformer_encoder.layers:
        this_layer_attn = layer.self_attn(x,x,x, average_attn_weights=False)[1]
        attentions.append(this_layer_attn.cpu())
        x = layer(x)
    return attentions

def viz(start, end, attentions, exampleTokenized, threshold=0.005, layer=1, heads=[]):
    m = 1/threshold
    tokens = exampleTokenized[start:end]
    att_crop = [x[:, :, start:end, start:end]*m for x in attentions]
    head_view(att_crop, tokens, prettify_tokens=False, heads=heads, layer=layer)

def report_where_attends(seq_1, seq_2, token_location, tokenized_input):
    # TODO: somewhat broken?
    msg = ""
    token = tokenized_input[token_location]
    where_attends_seq_2 = np.where(seq_1 == token_location)[0]
    if len(where_attends_seq_2) > 0:
        counterpart_tokens = list(set([tokenized_input[x] for x in seq_2[where_attends_seq_2].tolist()]))
        if token in counterpart_tokens:
            counterpart_tokens.remove(token)
        if counterpart_tokens: # TODO: this part reports wrong results
            msg = f"\t  It has connections with tokens at following locations and values:\n"
            msg += f"\t\tLocated: {where_attends_seq_2}\n"
            msg += f"\t\tValues: {counterpart_tokens}\n"
    return msg

def analyze_attentions(attentions, tokenized_input, threshold=0.005, diff=20, most_common=5, types=["proximity", "frequency"]):
    for layer_nr, layer in enumerate(attentions):
        idxs = np.where(layer > threshold)
        if len(idxs[0]) > 0:
            print(f"\n[!] Total {len(idxs[0])} tokens has strong activations at layer {layer_nr} with threshold: {threshold}!")
            heads = idxs[1].tolist()
            attn_seq_1 = idxs[2]
            attn_seq_2 = idxs[3]

            if "proximity" in types:
                # PROXIMITY CHECK
                print("[*] Performing a token attention proximity check ... ")
                for j, (a1, a2) in enumerate(zip(attn_seq_1, attn_seq_2)):
                    # check if difference between a1 and a2 is below threshold
                    if abs(a1 - a2) < diff:
                        which_heads = heads[j]
                        print("\tThese two close sequence tokens has strong activations:", a1, a2, f"at layer {layer_nr} head", which_heads)
                        token_at_a1 = tokenized_input[a1]
                        token_at_a2 = tokenized_input[a2]
                        print(f"\t\tToken at location {a1} -> {token_at_a1}")
                        print(f"\t\tToken at location {a2} -> {token_at_a2}")
                        aa = sorted([a1, a2])
                        a1, a2 = aa[0]-5, aa[1]+5
                        vizString = f"viz({a1}, {a2}, attentions, tokenized_input, layer={layer_nr}, heads=[{which_heads}])"
                        print("Calling:", vizString)
                        viz(a1, a2, attentions, tokenized_input, layer=layer_nr, heads=[which_heads])
            if "count" in types or "frequency" in types:
                # COUNTER CHECK
                print("[*] Performing a token frequency check ... ")
                c = attn_seq_2.tolist()
                c.extend((attn_seq_1.tolist()))
                c = Counter(c).most_common(most_common)        
                for token_location, token_frequency in c:
                    appear_in_heads = [heads[i] for i in np.where(attn_seq_1 == token_location)[0].tolist()]
                    appear_in_heads.extend([heads[i] for i in np.where(attn_seq_2 == token_location)[0].tolist()])
                    appear_in_heads = list(set(appear_in_heads))

                    token_value = tokenized_input[token_location]
                    msg1 = report_where_attends(attn_seq_1, attn_seq_2, token_location, tokenized_input)
                    msg2 = report_where_attends(attn_seq_2, attn_seq_2, token_location, tokenized_input)
                    if msg1 or msg2:
                        msg = f"\n\t> Token '{token_value}' location in sequence: {token_location}\n"
                        msg += f"\t  It appears {token_frequency} times, in layer {layer_nr}, heads: {appear_in_heads}\n"
                        msg += msg1 + msg2
                        print(msg)
                    
def get_attention_report(exampleFile, model, maxLen=512, random_seed=42, diff=20):
    with open(exampleFile) as f:
        example = json.load(f)

    extractor = PEDynamicFeatureExtractor()
    exampleProcessed = extractor.filter_and_normalize_report(example[0])
    
    tokenizer = JSONTokenizerBPE(model_path=os.path.join(ROOT, r"nebula\objects\speakeasy_BPE_50000.model"))

    exampleTokenized = tokenizer.tokenize(exampleProcessed)[0]
    exampleEncoded = tokenizer.encode(exampleProcessed)[0]
    x = torch.Tensor(exampleEncoded).long().reshape(1,-1).cuda()

    logit_bad = model(x[:, :maxLen]).cpu()
    prob_bad = torch.sigmoid(logit_bad)
    print("[!] Model scores on this sample -- logit:", logit_bad[0][0].numpy(), " and probability:", prob_bad[0][0].numpy())

    clear_cuda_cache()
    set_random_seed(random_seed)
    attentions = get_attn(x, model)
    analyze_attentions(attentions, exampleTokenized, types=["proximity"], threshold=0.004, diff=diff)
    analyze_attentions(attentions, exampleTokenized, types=["frequency"], most_common=10)
    return attentions, exampleTokenized

In [3]:
# done three folds on that run
model_file = r"evaluation\crossValidation\BPE\Transformer_longRun_30_epochs\trainingFiles\trainingFiles_1674502174-model.torch"
#model_file = r"evaluation\crossValidation\BPE\Transformer_longRun_30_epochs\trainingFiles\trainingFiles_1674504880-model.torch"
#model_file = r"evaluation\crossValidation\BPE\Transformer_longRun_30_epochs\trainingFiles\trainingFiles_1674507543-model.torch"
state_dict = torch.load(os.path.join(ROOT, model_file))

modelArch = {
    "vocabSize": 50000,  # size of vocabulary
    "maxLen": 512,  # maximum length of the input sequence
    "dModel": 64,  # embedding & transformer dimension
    "nHeads": 8,  # number of heads in nn.MultiheadAttention
    "dHidden": 256,  # dimension of the feedforward network model in nn.TransformerEncoder
    "nLayers": 2,  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
    "numClasses": 1, # binary classification
    "hiddenNeurons": [64],
    "layerNorm": False,
    "dropout": 0.3
}
model = TransformerEncoderModel(**modelArch)
model.load_state_dict(state_dict)
_ = model.cuda()

In [18]:
report_root = os.path.join(ROOT, r"data\data_raw\windows_emulation_trainset\report_backdoor")
exampleFile = os.path.join(report_root, "0009064322cdc719a82317553b805cbbc64230a9212d3b8aad1ea1b78d3bf10a.json")
# https://www.virustotal.com/gui/file/0009064322cdc719a82317553b805cbbc64230a9212d3b8aad1ea1b78d3bf10a
attentions, tokenized_input = get_attention_report(exampleFile, model, maxLen=512, random_seed=1492, diff=10)

[!] Model scores on this sample -- logit: 26.689674  and probability: 1.0

[!] Total 319 tokens has strong activations at layer 1 with threshold: 0.004!
[*] Performing a token attention proximity check ... 
	These two close sequence tokens has strong activations: 678 685 at layer 1 head 0
		Token at location 678 -> ▁0x100
		Token at location 685 -> ▁0xfeee0001
Calling: viz(673, 690, attentions, tokenized_input, layer=1, heads=[0])


<IPython.core.display.Javascript object>

	These two close sequence tokens has strong activations: 1123 1128 at layer 1 head 6
		Token at location 1123 -> ▁kernel32.
		Token at location 1128 -> leavecriticalsection
Calling: viz(1118, 1133, attentions, tokenized_input, layer=1, heads=[6])


<IPython.core.display.Javascript object>


[!] Total 64 tokens has strong activations at layer 1 with threshold: 0.005!
[*] Performing a token frequency check ... 

	> Token '▁0x1' location in sequence: 878
	  It appears 3 times, in layer 1, heads: [5]
	  It has connections with tokens at following locations and values:
		Located: [31 32 33]
		Values: ['▁0x46f0', 'wsprintfw']



In [20]:
viz(30,40, attentions, tokenized_input, layer=1, heads=[5])

<IPython.core.display.Javascript object>

## Step-by-step

In [5]:
with open(exampleFile) as f:
    example = json.load(f)
print(example[0].keys())
print(example[0]['file_access'][-2:])

extractor = PEDynamicFeatureExtractor()
exampleProcessed = extractor.filter_and_normalize_report(example[0])
exampleProcessed['file_access']

dict_keys(['ep_type', 'start_addr', 'ep_args', 'apihash', 'apis', 'ret_val', 'error', 'file_access', 'dynamic_code_segments', 'dropped_files'])
[{'event': 'create', 'path': 'Cd.exedows\\temp\\HGDraw.dll', 'open_flags': ['CREATE_ALWAYS'], 'access_flags': ['GENERIC_WRITE']}, {'event': 'write', 'path': 'Cd.exedows\\temp\\HGDraw.dll', 'data': 'TVqQAAMAAAAEAAAA//8AALgAAAAAAAAAQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA8AAAAA4fug4AtAnNIbgBTM0hVGhpcyBwcm9ncmFtIGNhbm5vdCBiZSBydW4gaW4gRE9TIG1vZGUuDQ0KJAAAAAAAAABJLe9jDUyBMA1MgTANTIEwYjoqMCBMgTBiOh8wG0yBMGI6KzCOTIEwBDQSMAJMgTANTIAwqEyBMGI6LjAfTIEwYjobMAxMgTBiOhwwDEyBMFJpY2gNTIEwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAUEUAAEwBAwBWBhxSAAAAAAAAAADgAAIBCwEKAACQAQAAhgQAAAAAAFrFAAAAEAAAAKABAAAAQAAAEAAAAAIAAAUAAQAAAAAABQABAAAAAAAA4AYAAAQAANnPBQACAECBAAAQAAAQAAAAABAAABAAAAAAAAAQAAAAAAAAAAAAAADcFAIAjAAAAABgBgC6YgAAAAAAAAAAAAAAAAAAAAAAAABABgB0DwAAQKIBABwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAudGV4dA

[{'event': 'create', 'path': '<drive>\\windows\\temp\\golfinfo.ini'},
 {'event': 'create', 'path': '<drive>\\windows\\temp\\golfset.ini'},
 {'event': 'create', 'path': '<drive>\\windows\\temp\\golfinfo.ini'},
 {'event': 'write', 'path': '<drive>\\windows\\temp\\golfinfo.ini'},
 {'event': 'open', 'path': '<drive>\\windows\\system32\\<sha256>'},
 {'event': 'read', 'path': '<drive>\\windows\\system32\\<sha256>'},
 {'event': 'create', 'path': 'cd.exedows\\temp\\hgdraw.dll'},
 {'event': 'write', 'path': 'cd.exedows\\temp\\hgdraw.dll'}]

In [6]:
tokenizer = JSONTokenizerBPE(model_path=os.path.join(ROOT, r"nebula\objects\speakeasy_BPE_50000.model"))

exampleTokenized = tokenizer.tokenize(exampleProcessed)[0]
exampleEncoded = tokenizer.encode(exampleProcessed)[0]

print(exampleTokenized[0:20])
print(exampleEncoded[0:20])
x = torch.Tensor(exampleEncoded).long().reshape(1,-1).cuda()

['▁file', '_', 'access', '▁create', '▁<', 'drive', '>', '▁windows', '▁temp', '▁golfinfo', '.', 'ini', '▁create', '▁<', 'drive', '>', '▁windows', '▁temp', '▁golfset', '.']
[1269, 49982, 990, 371, 738, 846, 49988, 278, 420, 1354, 49963, 309, 371, 738, 846, 49988, 278, 420, 2311, 49963]


In [7]:
logit_bad = model(x[:, :512]).cpu()
prob_bad = torch.sigmoid(logit_bad)
print("Model scores on this sample -- logit:", logit_bad[0][0].numpy(), " and probability:", prob_bad[0][0].numpy())

Model scores on this sample -- logit: 20.7325  and probability: 1.0


### Getting attention activations

In [8]:
clear_cuda_cache()
set_random_seed(42)
attentions = get_attn(x, model)

In [9]:
analyze_attentions(attentions, exampleTokenized, types=["count"])


[!] Total 73 tokens has strong activations at layer 1 with threshold: 0.005!
[*] Performing a token frequency check ... 

	> Token '▁kernel32.' location in sequence: 466
	  It appears 6 times, in layer 1, heads: [6]
	  It has connections with tokens at following locations and values:
		Located: [37 38 39 40 41 42]
		Values: ['▁0x0', 'tlsgetvalue', '▁temp']



In [10]:
analyze_attentions(attentions, exampleTokenized, threshold=0.005, diff=30, types=["proximity"])


[!] Total 73 tokens has strong activations at layer 1 with threshold: 0.005!
[*] Performing a token attention proximity check ... 
	These two close sequence tokens has strong activations: 1554 1530 at layer 1 head 6
		Token at location 1554 -> ▁kernel32.
		Token at location 1530 -> ▁0x0
Calling: viz(1525, 1559, attentions, tokenized_input, layer=1, heads=[6])


<IPython.core.display.Javascript object>

In [11]:
viz(465, 475, attentions, exampleTokenized)

<IPython.core.display.Javascript object>

In [12]:
viz(60, 90, attentions, exampleTokenized)

<IPython.core.display.Javascript object>

In [13]:
viz(35, 45, attentions, exampleTokenized)

<IPython.core.display.Javascript object>